In [ ]:
#Python3
#install all the packages 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
image = mpimg.imread('test_images/solidWhiteRight.jpg')

print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)

In [ ]:
import math

def grayscale(img):
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    
    mask = np.zeros_like(img)   
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
           
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

def weighted_img(img, initial_img, a=0.7, b=1., g=0.):
    
    return cv2.addWeighted(initial_img, a, img, b, g)

In [ ]:
def x_val_point(line, y):
    
    m = slope(line)
    x1,y1,x2,y2 = line[0]
    return [int(1.0*(y-y1)/m +x1+.5), y]

def length(line):
    
    x1,y1,x2,y2 = line[0]
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def slope(line):
    
    x1,y1,x2,y2 = line[0]
    if x1==x2:
        return np.inf
    return 1.0*(y2-y1)/(x2-x1)

def wavg(v,w, rounded=True):
    
    x= np.dot(v,w)/np.sum(w)
    if rounded:
        return int(x+.5)
    else:
        return x

def filter_lines_by_slope(lines,xs,low=.5):
    
    new_lines=[]
    
    for line in lines:
        m=slope(line)
        
        if m==np.inf:
            new_lines.append(line)
        elif m < -1.0*low and line[0][0] <= xs/2 and line[0][2] <= xs/2:
            new_lines.append(line)
        elif m > low and line[0][0] >= xs/2 and line[0][2] >= xs/2:
            new_lines.append(line)
          
    return new_lines

def bounding_triangle(lines,ys):
    
    sizes = np.array(list(map(length,lines)))
    xint=np.array(list(map(lambda x: x_val_point(x,ys)[0], lines)))
    
    wcenter = wavg(xint,sizes, rounded=False)

    a= [wavg(xint[xint>=wcenter],sizes[xint >= wcenter]),ys]
    b= [wavg(xint[xint<=wcenter],sizes[xint <= wcenter]),ys]

    y_max = int(ys*.6)

    xint = np.array(list(map(lambda x: x_val_point(x,y_max)[0], lines)))

    c= [wavg(xint,sizes),y_max]

    return np.array([a,b,c])

In [ ]:
def plot_results(results,N=5):

    n = N
    m = int(n/N)+1
    ys, xs, _ = results[0][0].shape
    plt.figure(figsize=(n*xs/200,m*ys/200))

    for i,r in enumerate(results):
        plt.subplot(m,n,i+1)
        if r[1]==1:
            plt.imshow(r[0],cmap='gray')
        else:
            plt.imshow(r[0])

In [ ]:
def lane_mask(image):
    
    lowery = np.array([175,175,0])
    upper= np.array([255,255,150])
    yellow = cv2.inRange(image, lowery, upper)
    
    lowery = np.array([195,195,195])
    upper= np.array([255,255,255])
    white = cv2.inRange(image, lowery, upper)
  
    gray = cv2.bitwise_or(yellow,white)
    
    return gray

In [ ]:
FRAME_MEMORY=[]
LINE_MEMORY=[]
def process_image(image):
    try:
        return process_img(image)[-1][0]
    except:
        return image
    
def process_img(image):
    
    global FRAME_MEMORY
    global LINE_MEMORY
    
    N=5
    M=5
    
    if len(FRAME_MEMORY)==0:
        FRAME_MEMORY=[image]*N
        
    color=[255, 0, 0]
    frames=[(image,0)]
    
    ys, xs, _ = image.shape
    
    gray = lane_mask(image)
    for img in FRAME_MEMORY:
        gry = lane_mask(img)
        gray = cv2.bitwise_or(gray,gry)
    
    FRAME_MEMORY.pop(0)
    
    FRAME_MEMORY.append(image)
    
    
    frames.append((gray,1))
    gauss = gaussian_blur(gray, 7)
    can = canny(gauss, 40,120)
    
    y_max = ys*.575
    region = np.array([[[0,ys],[xs,ys],[xs/2,y_max]]],dtype=np.int32)
    can_region=region_of_interest(can,region)
    frames.append((can_region,1))
    
    try:
        himg, lines = hough_lines(can_region, 1,np.pi/180, 20, 50, 50)
        
        lines = filter_lines_by_slope(lines,xs)
        blank=np.zeros((ys, xs, 3), dtype=np.uint8)
        draw_lines(blank,lines)
        frames.append((blank,0))
    except:
        if LINE_MEMORY==[]:
            return frames.append((image,0))
        
    if LINE_MEMORY==[]:
        LINE_MEMORY=[lines]*M
        
    LINE_MEMORY.pop(0)
    
    LINE_MEMORY.append(lines)
        
    left=[]
    right=[]
    for line_set in LINE_MEMORY:
        for line in line_set:
            if slope(line) < 0:
                left.append(line)
            else:
                right.append(line)
                
    blank=np.zeros((ys, xs, 3), dtype=np.uint8)
    for lane in [left,right]:
        if len(lane) == 0:
            continue
            
        points=bounding_triangle(lane,ys)
        pvc=cv2.convexHull(points)
        
       
        cv2.drawContours(blank,[pvc],0,color,-1)
        cv2.drawContours(image,[pvc],0,color,2)
        
            
    result = weighted_img(image, blank)
    frames.append((result,0))
    
    return frames

In [ ]:
import os
os.listdir("test_images/")

In [ ]:
frames =['f90.jpg','f110.jpg','f125.jpg','f140.jpg']

#can be done in two ways...
'''
for f in frames:
    FRAME_MEMORY=[]
    LINE_MEMORY=[]
    image = mpimg.imread('solid_white/%s'%f)
    results = process_img(image)
    plot_results(results)
    plt.title(f)
    
frames =['f90.jpg','f110.jpg','f125.jpg','f140.jpg']

for f in frames:
    FRAME_MEMORY=[]
    LINE_MEMORY=[]
    image = mpimg.imread('challenge_frames/%s'%f)
    results = process_img(image)
    plot_results(results)
    plt.title(f)
'''

for f in os.listdir("test_images/"):
    FRAME_MEMORY=[]
    LINE_MEMORY=[]
    if not f.endswith('jpg'):
        continue
        
    image = mpimg.imread('test_images/%s'%f)
    results = process_img(image)
    plot_results(results)


In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
i=0
def f(x):
    global i
    im=cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    cv2.imwrite('solid_white/f%d.jpg'%i,im)
    i+=1
    return x


FRAME_MEMORY=[]
LINE_MEMORY=[]
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) 
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">a
</video> 
""".format(white_output))

#click the play button to play the video

In [ ]:
FRAME_MEMORY=[]
LINE_MEMORY=[]
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))